In [12]:
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

### 2015 ~ 2020년도 야구 데이터 가져오기

In [13]:
filenames=['C:/workspace/projects/data/baseball_project/baseball_' + str(x) + '.csv' for x in list(range(2015,2021))]

data = pd.DataFrame()
for filename in tqdm(filenames):
    temp = pd.read_csv(filename)
    data = pd.concat([data,temp])

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 116.23it/s]


In [14]:
baseball_data = data.copy()

### 전처리 함수

In [15]:
def preprocessing(df):
    # 무승부 제거
    df = df[df['win']!=0.5]

    # 넥센 팀 명이 키움으로 변경됨
    df['T_ID'] = df['T_ID'].replace('NE','WO')
    df['VS_T_ID'] = df['VS_T_ID'].replace('NE','WO')

    # 팀 명 라벨링
    le = LabelEncoder()
    le.fit(df['T_ID'])
    df.loc[:,'T_ID'] = le.transform(df['T_ID'])
    df.loc[:,'VS_T_ID'] = le.transform(df['VS_T_ID'])
    
    return df

### 1. RandomForest 모델 학습

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier

In [17]:
baseball_data = preprocessing(baseball_data)

### 데이터 셋 나누기

In [18]:
train_data = baseball_data[['RUN','AB','PA','OOO','T_ID','VS_T_ID','OBP']]
train_target = baseball_data['win']

In [19]:
train_x, test_x, train_y, test_y = train_test_split(train_data, train_target, test_size = 0.2, random_state = 42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(6817, 7) (1705, 7) (6817,) (1705,)


### GridSearchCV를 이용하여 하이퍼파라미터 튜닝

In [20]:
params={
    'max_depth':[4,8,12,16,20,24],
    'min_samples_leaf':[6,9,12,15,18,21],
    'min_samples_split':[8,16,24,32,40,48,56]
}

rfc = RandomForestClassifier(n_estimators=50,random_state=42,n_jobs=-1)
grid_cv=GridSearchCV(rfc, param_grid=params,cv=5,n_jobs=-1)
grid_cv.fit(train_x,train_y)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_estimators=50, n_jobs=-1,
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [4, 8, 12, 16, 20, 24],
                         'min_samples_leaf': [6, 9, 12, 15, 18, 21],
                         'min_samples_split': [8, 16, 24, 32, 40, 48, 56]})

In [21]:
print('최적 하이퍼 파라미터:\n',grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:
 {'max_depth': 24, 'min_samples_leaf': 6, 'min_samples_split': 16}
최고 예측 정확도: 0.8109


### 정확도 측정

In [22]:
from sklearn.metrics import accuracy_score

rfc_model = grid_cv.best_estimator_
pred = rfc_model.predict(test_x)
print('예측 정확도: {0:.4f}'.format(accuracy_score(test_y, pred)))

예측 정확도: 0.8018


### RandomForest 모델 저장

In [23]:
pickle.dump(rfc_model, open('C:/workspace/projects/data/baseball_project/model/random_forest.pkl', 'wb'))

### 2. XGBoost 모델

In [24]:
import xgboost as xgb

In [25]:
baseball_data = data.copy()

# 데이터 전처리
baseball_data = preprocessing(baseball_data)

In [26]:
X_train, X_test, y_train, y_test=train_test_split(train_data, train_target,
                                         test_size=0.2, random_state=42 )

X_tr, X_val, y_tr, y_val= train_test_split(X_train, y_train, test_size=0.1, random_state=42 )

In [27]:
dtr = xgb.DMatrix(data=X_tr, label=y_tr)
dval = xgb.DMatrix(data=X_val, label=y_val)
dtest = xgb.DMatrix(data=X_test , label=y_test)

In [28]:
params = { 'max_depth':3,
          'eta': 0.05,
          'objective':'binary:logistic',
          'eval_metric':'logloss'
         }
num_rounds = 400

In [29]:

eval_list = [(dtr,'train'),(dval,'eval')]

xgb_model = xgb.train(params = params , dtrain=dtr , num_boost_round=num_rounds ,\
                      early_stopping_rounds=50, evals=eval_list )

[0]	train-logloss:0.67431	eval-logloss:0.67492
[1]	train-logloss:0.65728	eval-logloss:0.65844
[2]	train-logloss:0.64146	eval-logloss:0.64276
[3]	train-logloss:0.62709	eval-logloss:0.62868
[4]	train-logloss:0.61415	eval-logloss:0.61649
[5]	train-logloss:0.60209	eval-logloss:0.60461
[6]	train-logloss:0.59066	eval-logloss:0.59264
[7]	train-logloss:0.58021	eval-logloss:0.58194
[8]	train-logloss:0.57084	eval-logloss:0.57304
[9]	train-logloss:0.56189	eval-logloss:0.56369
[10]	train-logloss:0.55362	eval-logloss:0.55586
[11]	train-logloss:0.54622	eval-logloss:0.54888
[12]	train-logloss:0.53902	eval-logloss:0.54143
[13]	train-logloss:0.53206	eval-logloss:0.53516
[14]	train-logloss:0.52581	eval-logloss:0.52893
[15]	train-logloss:0.51981	eval-logloss:0.52397
[16]	train-logloss:0.51400	eval-logloss:0.51883
[17]	train-logloss:0.50900	eval-logloss:0.51400
[18]	train-logloss:0.50404	eval-logloss:0.50935
[19]	train-logloss:0.49926	eval-logloss:0.50550
[20]	train-logloss:0.49431	eval-logloss:0.50125
[2

[170]	train-logloss:0.38421	eval-logloss:0.41873
[171]	train-logloss:0.38411	eval-logloss:0.41868
[172]	train-logloss:0.38402	eval-logloss:0.41859
[173]	train-logloss:0.38382	eval-logloss:0.41851
[174]	train-logloss:0.38376	eval-logloss:0.41854
[175]	train-logloss:0.38370	eval-logloss:0.41854
[176]	train-logloss:0.38351	eval-logloss:0.41866
[177]	train-logloss:0.38342	eval-logloss:0.41869
[178]	train-logloss:0.38334	eval-logloss:0.41863
[179]	train-logloss:0.38305	eval-logloss:0.41886
[180]	train-logloss:0.38291	eval-logloss:0.41890
[181]	train-logloss:0.38282	eval-logloss:0.41885
[182]	train-logloss:0.38275	eval-logloss:0.41885
[183]	train-logloss:0.38269	eval-logloss:0.41887
[184]	train-logloss:0.38253	eval-logloss:0.41866
[185]	train-logloss:0.38240	eval-logloss:0.41842
[186]	train-logloss:0.38223	eval-logloss:0.41833
[187]	train-logloss:0.38206	eval-logloss:0.41843
[188]	train-logloss:0.38178	eval-logloss:0.41866
[189]	train-logloss:0.38165	eval-logloss:0.41875
[190]	train-logloss:

### 정확도 측정

In [30]:
pred_probs = xgb_model.predict(dtest)
preds = [ 1 if x > 0.5 else 0 for x in pred_probs ]
accuracy_score(y_test , preds)

0.8

### XGBoost 모델 저장

In [31]:
pickle.dump(xgb_model, open('C:/workspace/projects/data/baseball_project/model/xgboost.pkl', 'wb'))

### 3. GBM 모델 학습

In [32]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import uniform, randint
from sklearn.metrics import accuracy_score

In [33]:
baseball_data = data.copy()

# 데이터 전처리
baseball_data = preprocessing(baseball_data)

### 데이터 셋 나누기

In [34]:
train_data = baseball_data[['RUN','AB','PA','OOO','T_ID','VS_T_ID','OBP']]
train_target = baseball_data['win']

train_x, test_x, train_y, test_y = train_test_split(train_data, train_target, test_size = 0.2, random_state = 42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(6817, 7) (1705, 7) (6817,) (1705,)


### RandomSearchCV로 하이퍼파라미터 튜닝

In [35]:
start_time = time.time()

params = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(1, 20),
    'min_samples_leaf': randint(2, 20),
    'min_samples_split': randint(2, 20),
    'learning_rate': uniform(0.001, 0.2)
}

gb_clf = GradientBoostingClassifier(random_state = 0)
random_cv = RandomizedSearchCV(gb_clf, params, n_iter=100, n_jobs=-1)
random_cv.fit(train_x, train_y)
best_rs_gbm = random_cv.best_estimator_

print('최적의 파라미터 :',random_cv.best_params_)
print('최고의 예측 정확도 :{0:.4f}'.format(random_cv.best_score_))
print('RandomSearchCV 수행 시간: {0:.1f} 초'.format(time.time() - start_time))

최적의 파라미터 : {'learning_rate': 0.0898619345605377, 'max_depth': 2, 'min_samples_leaf': 11, 'min_samples_split': 2, 'n_estimators': 144}
최고의 예측 정확도 :0.8118
RandomSearchCV 수행 시간: 228.0 초


In [36]:
gb_pred3 = best_rs_gbm.predict(test_x)
gb_accuracy3 = accuracy_score(test_y, gb_pred3)
print('예측 정확도: {0:.4f}'.format(gb_accuracy3))

예측 정확도: 0.7977


### 모델 저장

In [37]:
pickle.dump(best_rs_gbm, open('C:/workspace/projects/data/baseball_project/model/GBM.pkl', 'wb'))

### 4. LightGBM 모델 학습

In [38]:
from hyperopt import hp
from sklearn.model_selection import KFold
from hyperopt import fmin, tpe, Trials
from sklearn.metrics import roc_auc_score

from lightgbm import LGBMClassifier

In [39]:
baseball_data = data.copy()

# 데이터 전처리
baseball_data = preprocessing(baseball_data)

### 데이터 셋 나누기

In [40]:
X_features = baseball_data[['RUN','AB','PA','OOO','T_ID','VS_T_ID','OBP']]
y_labels = baseball_data['win']

X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels,
                                                    test_size=0.2, random_state=42)

X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train,
                                                    test_size=0.3, random_state=42)

In [41]:
lgbm_clf = LGBMClassifier(n_estimators=500)

eval_set=[(X_tr, y_tr), (X_val, y_val)]
lgbm_clf.fit(X_tr, y_tr, early_stopping_rounds=100, eval_metric="auc", eval_set=eval_set)

lgbm_roc_score = roc_auc_score(y_test, lgbm_clf.predict_proba(X_test)[:,1])
print('ROC AUC: {0:.4f}'.format(lgbm_roc_score))

[1]	training's auc: 0.889484	training's binary_logloss: 0.648467	valid_1's auc: 0.876146	valid_1's binary_logloss: 0.649983
[2]	training's auc: 0.892835	training's binary_logloss: 0.611682	valid_1's auc: 0.875417	valid_1's binary_logloss: 0.615299
[3]	training's auc: 0.896188	training's binary_logloss: 0.58093	valid_1's auc: 0.877923	valid_1's binary_logloss: 0.586146
[4]	training's auc: 0.901703	training's binary_logloss: 0.554226	valid_1's auc: 0.883087	valid_1's binary_logloss: 0.562313
[5]	training's auc: 0.903558	training's binary_logloss: 0.531865	valid_1's auc: 0.88396	valid_1's binary_logloss: 0.541698
[6]	training's auc: 0.906394	training's binary_logloss: 0.512064	valid_1's auc: 0.888785	valid_1's binary_logloss: 0.522491
[7]	training's auc: 0.907854	training's binary_logloss: 0.494961	valid_1's auc: 0.889461	valid_1's binary_logloss: 0.507643
[8]	training's auc: 0.908807	training's binary_logloss: 0.480242	valid_1's auc: 0.890111	valid_1's binary_logloss: 0.494364
[9]	traini

[70]	training's auc: 0.953647	training's binary_logloss: 0.292093	valid_1's auc: 0.895772	valid_1's binary_logloss: 0.408236
[71]	training's auc: 0.954106	training's binary_logloss: 0.291003	valid_1's auc: 0.895706	valid_1's binary_logloss: 0.408379
[72]	training's auc: 0.954598	training's binary_logloss: 0.289805	valid_1's auc: 0.895596	valid_1's binary_logloss: 0.408645
[73]	training's auc: 0.95521	training's binary_logloss: 0.288456	valid_1's auc: 0.895295	valid_1's binary_logloss: 0.40935
[74]	training's auc: 0.955611	training's binary_logloss: 0.287608	valid_1's auc: 0.895396	valid_1's binary_logloss: 0.409294
[75]	training's auc: 0.956014	training's binary_logloss: 0.286604	valid_1's auc: 0.895441	valid_1's binary_logloss: 0.409188
[76]	training's auc: 0.95641	training's binary_logloss: 0.285606	valid_1's auc: 0.895741	valid_1's binary_logloss: 0.409034
[77]	training's auc: 0.956919	training's binary_logloss: 0.284379	valid_1's auc: 0.895677	valid_1's binary_logloss: 0.409139
[78

In [42]:
lgbm_search_space = {'num_leaves': hp.quniform('num_leaves', 32, 64, 1),
                     'max_depth': hp.quniform('max_depth', 100, 160, 1),
                     'min_child_samples': hp.quniform('min_child_samples', 60, 100, 1),
                     'subsample': hp.uniform('subsample', 0.7, 1),
                     'learning_rate': hp.uniform('learning_rate', 0.01, 0.2)

                    }

In [43]:
def objective_func(search_space):
    lgbm_clf =  LGBMClassifier(n_estimators=100, num_leaves=int(search_space['num_leaves']),
                               max_depth=int(search_space['max_depth']),
                               min_child_samples=int(search_space['min_child_samples']), 
                               subsample=search_space['subsample'],
                               learning_rate=search_space['learning_rate'])
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = []
    
    # 3개 k-fold방식 적용 
    kf = KFold(n_splits=3)
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index값으로 학습과 검증 데이터 세트 분리 
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]

        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행. 
        lgbm_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="auc",
           eval_set=[(X_tr, y_tr), (X_val, y_val)])

        # 1로 예측한 확률값 추출후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음.
        score = roc_auc_score(y_val, lgbm_clf.predict_proba(X_val)[:, 1]) 
        roc_auc_list.append(score)
    
    # 3개 k-fold로 계산된 roc_auc값의 평균값을 반환하되, 
    # HyperOpt는 목적함수의 최소값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환.
    return -1*np.mean(roc_auc_list)

In [44]:
trials = Trials()

# fmin()함수를 호출. max_evals지정된 횟수만큼 반복 후 목적함수의 최소값을 가지는 최적 입력값 추출. 
best = fmin(fn=objective_func, space=lgbm_search_space, algo=tpe.suggest,
            max_evals=50, # 최대 반복 횟수를 지정합니다.
            trials=trials, rstate=np.random.default_rng(seed=30))

print('best:', best)

[1]	training's auc: 0.885698	training's binary_logloss: 0.682503	valid_1's auc: 0.880902	valid_1's binary_logloss: 0.683167
[2]	training's auc: 0.886786	training's binary_logloss: 0.672503	valid_1's auc: 0.883041	valid_1's binary_logloss: 0.673184
[3]	training's auc: 0.887793	training's binary_logloss: 0.662923	valid_1's auc: 0.884154	valid_1's binary_logloss: 0.663628
[4]	training's auc: 0.8881	training's binary_logloss: 0.653769	valid_1's auc: 0.883503	valid_1's binary_logloss: 0.654492
[5]	training's auc: 0.888876	training's binary_logloss: 0.645009	valid_1's auc: 0.884484	valid_1's binary_logloss: 0.645777
[6]	training's auc: 0.888969	training's binary_logloss: 0.636631	valid_1's auc: 0.88412	valid_1's binary_logloss: 0.637461
[7]	training's auc: 0.888652	training's binary_logloss: 0.628652	valid_1's auc: 0.884601	valid_1's binary_logloss: 0.629513
[8]	training's auc: 0.888819	training's binary_logloss: 0.620965	valid_1's auc: 0.88494	valid_1's binary_logloss: 0.62184
[9]	training'

[66]	training's auc: 0.907243	training's binary_logloss: 0.424193	valid_1's auc: 0.894316	valid_1's binary_logloss: 0.438728
[67]	training's auc: 0.907429	training's binary_logloss: 0.422884	valid_1's auc: 0.894411	valid_1's binary_logloss: 0.437563
[68]	training's auc: 0.907655	training's binary_logloss: 0.421511	valid_1's auc: 0.894651	valid_1's binary_logloss: 0.436392
[69]	training's auc: 0.907846	training's binary_logloss: 0.420183	valid_1's auc: 0.894775	valid_1's binary_logloss: 0.43526
[70]	training's auc: 0.908141	training's binary_logloss: 0.418843	valid_1's auc: 0.894907	valid_1's binary_logloss: 0.434155
[71]	training's auc: 0.908429	training's binary_logloss: 0.417542	valid_1's auc: 0.895094	valid_1's binary_logloss: 0.433094
[72]	training's auc: 0.908654	training's binary_logloss: 0.416263	valid_1's auc: 0.895288	valid_1's binary_logloss: 0.431957
[73]	training's auc: 0.908872	training's binary_logloss: 0.415035	valid_1's auc: 0.895457	valid_1's binary_logloss: 0.430977
[

[31]	training's auc: 0.904634	training's binary_logloss: 0.496731	valid_1's auc: 0.878815	valid_1's binary_logloss: 0.514903
[32]	training's auc: 0.904998	training's binary_logloss: 0.493165	valid_1's auc: 0.879152	valid_1's binary_logloss: 0.511811
[33]	training's auc: 0.905137	training's binary_logloss: 0.489797	valid_1's auc: 0.87913	valid_1's binary_logloss: 0.508932
[34]	training's auc: 0.905471	training's binary_logloss: 0.486456	valid_1's auc: 0.879303	valid_1's binary_logloss: 0.506047
[35]	training's auc: 0.905781	training's binary_logloss: 0.483223	valid_1's auc: 0.879515	valid_1's binary_logloss: 0.503284
[36]	training's auc: 0.906071	training's binary_logloss: 0.480108	valid_1's auc: 0.879736	valid_1's binary_logloss: 0.500613
[37]	training's auc: 0.906306	training's binary_logloss: 0.477114	valid_1's auc: 0.87976	valid_1's binary_logloss: 0.498187
[38]	training's auc: 0.906566	training's binary_logloss: 0.474178	valid_1's auc: 0.879827	valid_1's binary_logloss: 0.495726
[3

[96]	training's auc: 0.918403	training's binary_logloss: 0.383265	valid_1's auc: 0.886728	valid_1's binary_logloss: 0.429628
[97]	training's auc: 0.918581	training's binary_logloss: 0.38246	valid_1's auc: 0.886784	valid_1's binary_logloss: 0.429259
[98]	training's auc: 0.918781	training's binary_logloss: 0.381669	valid_1's auc: 0.886848	valid_1's binary_logloss: 0.428902
[99]	training's auc: 0.918939	training's binary_logloss: 0.380878	valid_1's auc: 0.88682	valid_1's binary_logloss: 0.428607
[100]	training's auc: 0.919105	training's binary_logloss: 0.380107	valid_1's auc: 0.886919	valid_1's binary_logloss: 0.428191
[1]	training's auc: 0.88771	training's binary_logloss: 0.682348	valid_1's auc: 0.876497	valid_1's binary_logloss: 0.683923
[2]	training's auc: 0.887928	training's binary_logloss: 0.672234	valid_1's auc: 0.876819	valid_1's binary_logloss: 0.674123
[3]	training's auc: 0.888331	training's binary_logloss: 0.662579	valid_1's auc: 0.876883	valid_1's binary_logloss: 0.664833
[4]	t

[61]	training's auc: 0.908943	training's binary_logloss: 0.428555	valid_1's auc: 0.893372	valid_1's binary_logloss: 0.447841
[62]	training's auc: 0.909187	training's binary_logloss: 0.426938	valid_1's auc: 0.89346	valid_1's binary_logloss: 0.446533
[63]	training's auc: 0.909393	training's binary_logloss: 0.425377	valid_1's auc: 0.893595	valid_1's binary_logloss: 0.445218
[64]	training's auc: 0.90959	training's binary_logloss: 0.423814	valid_1's auc: 0.893658	valid_1's binary_logloss: 0.444041
[65]	training's auc: 0.909821	training's binary_logloss: 0.422318	valid_1's auc: 0.893747	valid_1's binary_logloss: 0.442883
[66]	training's auc: 0.910058	training's binary_logloss: 0.420845	valid_1's auc: 0.893805	valid_1's binary_logloss: 0.441867
[67]	training's auc: 0.910256	training's binary_logloss: 0.419438	valid_1's auc: 0.893888	valid_1's binary_logloss: 0.440741
[68]	training's auc: 0.910434	training's binary_logloss: 0.418047	valid_1's auc: 0.893945	valid_1's binary_logloss: 0.439616
[6

...

[9]	training's auc: 0.905085	training's binary_logloss: 0.463115	valid_1's auc: 0.878621	valid_1's binary_logloss: 0.486032
[10]	training's auc: 0.905788	training's binary_logloss: 0.452529	valid_1's auc: 0.87942	valid_1's binary_logloss: 0.477068
[11]	training's auc: 0.906675	training's binary_logloss: 0.443017	valid_1's auc: 0.880325	valid_1's binary_logloss: 0.469067
[12]	training's auc: 0.908099	training's binary_logloss: 0.433846	valid_1's auc: 0.881343	valid_1's binary_logloss: 0.461968
[13]	training's auc: 0.908423	training's binary_logloss: 0.426735	valid_1's auc: 0.881617	valid_1's binary_logloss: 0.456504
[14]	training's auc: 0.909629	training's binary_logloss: 0.419302	valid_1's auc: 0.881871	valid_1's binary_logloss: 0.451435
[15]	training's auc: 0.910806	training's binary_logloss: 0.412633	valid_1's auc: 0.882677	valid_1's binary_logloss: 0.446737
[16]	training's auc: 0.912126	training's binary_logloss: 0.406324	valid_1's auc: 0.883556	valid_1's binary_logloss: 0.442583
[1

[7]	training's auc: 0.901585	training's binary_logloss: 0.4931	valid_1's auc: 0.888867	valid_1's binary_logloss: 0.50459
[8]	training's auc: 0.902469	training's binary_logloss: 0.47885	valid_1's auc: 0.889294	valid_1's binary_logloss: 0.491514
[9]	training's auc: 0.902962	training's binary_logloss: 0.466757	valid_1's auc: 0.88968	valid_1's binary_logloss: 0.480409
[10]	training's auc: 0.903593	training's binary_logloss: 0.456182	valid_1's auc: 0.889806	valid_1's binary_logloss: 0.471312
[11]	training's auc: 0.904781	training's binary_logloss: 0.446308	valid_1's auc: 0.890788	valid_1's binary_logloss: 0.462644
[12]	training's auc: 0.906127	training's binary_logloss: 0.437354	valid_1's auc: 0.89182	valid_1's binary_logloss: 0.454747
[13]	training's auc: 0.907327	training's binary_logloss: 0.429389	valid_1's auc: 0.892297	valid_1's binary_logloss: 0.448501
[14]	training's auc: 0.908189	training's binary_logloss: 0.422427	valid_1's auc: 0.893002	valid_1's binary_logloss: 0.442362
[15]	trai

[12]	training's auc: 0.892727	training's binary_logloss: 0.568001	valid_1's auc: 0.8871	valid_1's binary_logloss: 0.569798
[13]	training's auc: 0.893892	training's binary_logloss: 0.560584	valid_1's auc: 0.887333	valid_1's binary_logloss: 0.562846
[14]	training's auc: 0.894186	training's binary_logloss: 0.553698	valid_1's auc: 0.887463	valid_1's binary_logloss: 0.556124
[15]	training's auc: 0.894943	training's binary_logloss: 0.546847	valid_1's auc: 0.887715	valid_1's binary_logloss: 0.549837
[16]	training's auc: 0.895594	training's binary_logloss: 0.540452	valid_1's auc: 0.887663	valid_1's binary_logloss: 0.543945
[17]	training's auc: 0.896337	training's binary_logloss: 0.53421	valid_1's auc: 0.887749	valid_1's binary_logloss: 0.538095
 96%|█████████████████████████████████████████████  | 48/50 [00:33<00:01,  1.49trial/s, best loss: -0.8957887728199893]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[16]	training's auc: 0.91224	training's binary_logloss: 0.425094	valid_1's auc: 0.89326	valid_1's binary_logloss: 0.44776
[17]	training's auc: 0.913035	training's binary_logloss: 0.419112	valid_1's auc: 0.893296	valid_1's binary_logloss: 0.443352
[18]	training's auc: 0.913759	training's binary_logloss: 0.413712	valid_1's auc: 0.894117	valid_1's binary_logloss: 0.438931
[19]	training's auc: 0.914693	training's binary_logloss: 0.408403	valid_1's auc: 0.894622	valid_1's binary_logloss: 0.434914
[20]	training's auc: 0.915599	training's binary_logloss: 0.40361	valid_1's auc: 0.894997	valid_1's binary_logloss: 0.431469
[21]	training's auc: 0.916285	training's binary_logloss: 0.399052	valid_1's auc: 0.895078	valid_1's binary_logloss: 0.428606
[22]	training's auc: 0.916926	training's binary_logloss: 0.395148	valid_1's auc: 0.895155	valid_1's binary_logloss: 0.426168
[23]	training's auc: 0.917559	training's binary_logloss: 0.391342	valid_1's auc: 0.895947	valid_1's binary_logloss: 0.422942
[24]

### 정확도 측정

In [45]:
y_pred = lgbm_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8082111436950147


### 모델 저장

In [46]:
pickle.dump(lgbm_clf, open('C:/workspace/projects/data/baseball_project/model/LightGBM.pkl', 'wb'))